In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy download en_core_web_lg

2023-04-29 21:30:24.349941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
df = pd.read_csv('/gdrive/My Drive/IMDB Dataset.csv')
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


**Task 1: Data preprocessing**

In [ ]:
nlp = spacy.load("en_core_web_lg")

def normalize(review, lowercase, remove_stopwords):
  if lowercase:
    review = review.lower()
  doc = nlp(review)
  lemmatized = list()
  for token in doc:
    if not remove_stopwords or (remove_stopwords and not token.is_stop):
      lemmatized.append(token.lemma_)
  return " ".join(lemmatized)

df['review'] = df['review'].apply(normalize, lowercase=True, remove_stopwords=True)

In [ ]:
import re
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# to remove html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
df['review'] = df['review'].apply(remove_URL)
df['review'] = df['review'].apply(remove_html)

In [ ]:
print(df.shape)
print(df['sentiment'].value_counts())
df.head()

(50000, 2)
positive    25000
negative    25000
Name: sentiment, dtype: int64


,review,sentiment
0,reviewer mention watch 1 oz episode hook . rig...,positive
1,wonderful little production . the filming tech...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically family little boy ( jake ) think zom...,negative
4,"petter mattei "" love time money "" visually stu...",positive


In [ ]:
df.to_csv('IMDB Dataset Processed.csv',index=False)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and test sets with an 80-20 split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['sentiment'])

In [ ]:
print(train_df.shape)
print(train_df['sentiment'].value_counts())
train_df.head()

(40000, 2)
negative    20000
positive    20000
Name: sentiment, dtype: int64


,review,sentiment
41171,spoiler : the gettaway action movie . action !...,negative
30066,"see wicked little thing "" 8 film die "" horrorf...",negative
12298,""" enter fat dragon "" funny martial art movie o...",positive
3473,"despite sequel potent original , comical remak...",negative
19393,movie terrible . john wayne brutal actor time ...,negative


In [ ]:
print(test_df.shape)
print(test_df['sentiment'].value_counts())
test_df.head()

(10000, 2)
positive    5000
negative    5000
Name: sentiment, dtype: int64


,review,sentiment
31383,pitch black surprisingly good movie . fan vin ...,positive
39810,"bad korean movie ! plot ridiculously complex ,...",negative
14869,probably bother comment film disturb constant ...,positive
37081,film spicy little piece film - making sam full...,positive
35656,"' bride chucky ' "" return greatness "" series g...",positive


**Task 2: Lexicon-based sentiment analysis**

In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 11.5 MB/s eta 0:00:00


In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
df_v = df.copy()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Create an instance of the VADER SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to classify the sentiment of a given review using VADER
def classify_sentiment(review):
    scores = analyzer.polarity_scores(review)
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the classify_sentiment function to the 'review' column of the DataFrame
df_v['vader_sentiment'] = df_v['review'].apply(classify_sentiment)

In [ ]:
print(df_v.shape)
print(df_v['vader_sentiment'].value_counts())
df_v.head()

(50000, 3)
positive    33933
negative    15771
neutral       296
Name: vader_sentiment, dtype: int64


,review,sentiment,vader_sentiment
0,reviewer mention watch 1 oz episode hook . rig...,positive,negative
1,wonderful little production . the filming tech...,positive,positive
2,think wonderful way spend time hot summer week...,positive,positive
3,basically family little boy ( jake ) think zom...,negative,negative
4,"petter mattei "" love time money "" visually stu...",positive,positive


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate the accuracy of the model
accuracy = accuracy_score(df_v['sentiment'], df_v['vader_sentiment'])

# Calculate the precision of the model
precision = precision_score(df_v['sentiment'], df_v['vader_sentiment'], average='weighted')

# Calculate the recall of the model
recall = recall_score(df_v['sentiment'], df_v['vader_sentiment'], average='weighted')

# Calculate the F1 score of the model
f1 = f1_score(df_v['sentiment'], df_v['vader_sentiment'], average='weighted')

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.673
Precision: 0.704
Recall: 0.673
F1 Score: 0.664


**Task 3: Naive Bayes model for sentiment analysis**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm  # Import the tqdm library

# Define a function to train and evaluate a Naive Bayes model
def train_and_evaluate_model(vectorizer, n_features):
    # Vectorize the training and test sets using the specified vectorizer and # of features
    X_train = vectorizer.fit_transform(train_df['review'])
    X_test = vectorizer.transform(test_df['review'])

    # Extract the target variable (sentiment) from the training and test sets
    y_train = train_df['sentiment']
    y_test = test_df['sentiment']

    # Train a Naive Bayes model on the training set
    model = MultinomialNB()
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the evaluation metrics for the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Print the evaluation metrics for the model
    print(f'Vectorizer: {type(vectorizer).__name__}, # of features: {n_features}')
    print(f'Accuracy: {accuracy:.3f}')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1 Score: {f1:.3f}')

# Try different combinations of vectorizers and # of features
vectorizers = [CountVectorizer(binary=True), CountVectorizer(), TfidfVectorizer()]
n_features_list = [1000, 5000, 10000]

for vectorizer in vectorizers:
    for n_features in n_features_list:
        # Wrap the inner loop in a tqdm() call to add a progress bar
        for _ in tqdm(range(1)):
          train_and_evaluate_model(vectorizer, n_features)

100%|██████████| 1/1 [00:04<00:00,  4.65s/it]


Vectorizer: CountVectorizer, # of features: 1000
Accuracy: 0.857
Precision: 0.857
Recall: 0.857
F1 Score: 0.857


100%|██████████| 1/1 [00:05<00:00,  5.65s/it]


Vectorizer: CountVectorizer, # of features: 5000
Accuracy: 0.857
Precision: 0.857
Recall: 0.857
F1 Score: 0.857


100%|██████████| 1/1 [00:04<00:00,  4.20s/it]


...

Vectorizer: TfidfVectorizer, # of features: 1000
Accuracy: 0.862
Precision: 0.862
Recall: 0.862
F1 Score: 0.862


100%|██████████| 1/1 [00:04<00:00,  4.46s/it]


Vectorizer: TfidfVectorizer, # of features: 5000
Accuracy: 0.862
Precision: 0.862
Recall: 0.862
F1 Score: 0.862


100%|██████████| 1/1 [00:04<00:00,  4.57s/it]

Vectorizer: TfidfVectorizer, # of features: 10000
Accuracy: 0.862
Precision: 0.862
Recall: 0.862
F1 Score: 0.862


I have Used CountVectorizer(binary=True), CountVectorizer, TfidfVectorizer  with 1000 , 5000 , 10000 features with combinations respectively. irrespective with the feature count the accuracy, precision ,recall ,F1 is constant for these vectorizers CountVectorizer(binary=True), CountVectorizer, TfidfVectorizer.

for the vectorizer CountVectorizer(binary=True) and CountVectorizer  the accuracy is same i,e: 85.7%.

For Vectorizer TfidfVectorizer the accuracy is 86.2%.

By observing the above results we can say that TfidfVectorizer is performing well as the accuracy, precision , recall and F1 scores are good compared to all other vectorizer.

**Task 4: SVM model for sentiment analysis**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define a function to train and evaluate an SVM model
def train_and_evaluate_model(vectorizer, n_features):
    # Vectorize the training and test sets using the specified vectorizer and # of features
    X_train = vectorizer.fit_transform(train_df['review'])
    X_test = vectorizer.transform(test_df['review'])

    # Extract the target variable (sentiment) from the training and test sets
    y_train = train_df['sentiment']
    y_test = test_df['sentiment']

    # Train an SVM model on the training set
    model = LinearSVC()
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the evaluation metrics for the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Print the evaluation metrics for the model
    print(f'Vectorizer: {type(vectorizer).__name__}, # of features: {n_features}')
    print(f'Accuracy: {accuracy:.3f}')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1 Score: {f1:.3f}')

# Try different combinations of vectorizers and # of features
vectorizers = [CountVectorizer(binary=True), CountVectorizer(), TfidfVectorizer()]
n_features_list = [1000, 5000, 10000]

for vectorizer in vectorizers:
    for n_features in n_features_list:
        # Wrap the inner loop in a tqdm() call to add a progress bar
        for _ in tqdm(range(1)):
            train_and_evaluate_model(vectorizer, n_features)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
100%|██████████| 1/1 [00:10<00:00, 10.98s/it]


Vectorizer: CountVectorizer, # of features: 1000
Accuracy: 0.855
Precision: 0.855
Recall: 0.855
F1 Score: 0.855


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
100%|██████████| 1/1 [00:10<00:00, 10.57s/it]


Vectorizer: CountVectorizer, # of features: 5000
Accuracy: 0.855
Precision: 0.855
Recall: 0.855
F1 Score: 0.855


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
100%|██████████| 1/1 [00:10<00:00, 10.80s/it]


...

Vectorizer: TfidfVectorizer, # of features: 1000
Accuracy: 0.886
Precision: 0.886
Recall: 0.886
F1 Score: 0.886


100%|██████████| 1/1 [00:06<00:00,  6.66s/it]


Vectorizer: TfidfVectorizer, # of features: 5000
Accuracy: 0.886
Precision: 0.886
Recall: 0.886
F1 Score: 0.886


100%|██████████| 1/1 [00:05<00:00,  5.03s/it]

Vectorizer: TfidfVectorizer, # of features: 10000
Accuracy: 0.886
Precision: 0.886
Recall: 0.886
F1 Score: 0.886


I have Used CountVectorizer(binary=True), CountVectorizer, TfidfVectorizer  with 1000 , 5000 , 10000 features with combinations respectively.
irrespective with the feature count the accuracy, precision ,recall ,F1 is constant for these vectorizers CountVectorizer(binary=True), CountVectorizer, TfidfVectorizer.

for the vectorizer CountVectorizer(binary=True)  is 85.5% 

for the vectorizer CountVectorizer  the accuracy  i,e: 85.7%.

For Vectorizer TfidfVectorizer the accuracy is 88.6%.

By observing the above results we can say that TfidfVectorizer is performing well as the accuracy, precision , recall and F1 scores are good compared to all other vectorizer.

**Task 5: Deep Learning Models for Sentiment Analysis**

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['sentiment'])

# Define the maximum vocabulary size
vocab_size = 10000
max_length = 200

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_df['review'])
X_train = tokenizer.texts_to_sequences(train_df['review'])
X_val = tokenizer.texts_to_sequences(val_df['review'])
X_test = tokenizer.texts_to_sequences(test_df['review'])

# Pad the sequences
# max_length = max([len(review) for review in X_train])
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
# Extract the target variable (sentiment) from the training and test sets
y_train = train_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
y_val = val_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
y_test = test_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

(20480, 200)
(5120, 200)
(10000, 200)


In [ ]:
#model1
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Set the vocabulary size and sequence length
vocab_size = 10000
max_length = 200


# Define the model architecture  -1
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping conditions
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/10
640/640 [==============================] - 80s 120ms/step - loss: 0.3991 - accuracy: 0.8246 - val_loss: 0.3034 - val_accuracy: 0.8754
Epoch 2/10
640/640 [==============================] - 31s 48ms/step - loss: 0.2261 - accuracy: 0.9169 - val_loss: 0.3200 - val_accuracy: 0.8754
Epoch 3/10
640/640 [==============================] - 21s 32ms/step - loss: 0.1752 - accuracy: 0.9372 - val_loss: 0.3608 - val_accuracy: 0.8545
Epoch 4/10
313/313 [==============================] - 3s 8ms/step - loss: 0.4136 - accuracy: 0.8385
Test accuracy: 0.839


In [ ]:
# Predict the sentiment of the test set
y_pred = model.predict(X_test)
y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

313/313 [==============================] - 3s 8ms/step
Accuracy: 0.8385
Precision: 0.8686560662165106
Recall: 0.7976
F1-score: 0.8316129704931706


In [ ]:
#model2
# Load the pre-trained embeddings
embeddings_index = {}
with open(r'/gdrive/My Drive/Colab Notebooks/glove.6B/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


# Create an embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping conditions
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/10
640/640 [==============================] - 19s 21ms/step - loss: 0.5291 - accuracy: 0.7426 - val_loss: 0.4365 - val_accuracy: 0.8068
Epoch 2/10
640/640 [==============================] - 11s 17ms/step - loss: 0.4035 - accuracy: 0.8206 - val_loss: 0.3892 - val_accuracy: 0.8250
Epoch 3/10
640/640 [==============================] - 11s 18ms/step - loss: 0.3578 - accuracy: 0.8465 - val_loss: 0.3532 - val_accuracy: 0.8475
Epoch 4/10
640/640 [==============================] - 12s 19ms/step - loss: 0.3259 - accuracy: 0.8619 - val_loss: 0.3448 - val_accuracy: 0.8516
Epoch 5/10
640/640 [==============================] - 11s 18ms/step - loss: 0.3029 - accuracy: 0.8736 - val_loss: 0.3374 - val_accuracy: 0.8607
Epoch 6/10
640/640 [==============================] - 11s 18ms/step - loss: 0.2768 - accuracy: 0.8890 - val_loss: 0.3259 - val_accuracy: 0.8652
Epoch 7/10
640/640 [==============================] - 10s 16ms/step - loss: 0.2539 - accuracy: 0.8975 - val_loss: 0.3410 - val_accuracy:

In [ ]:
# Predict the sentiment of the test set
y_pred = model.predict(X_test)
y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

313/313 [==============================] - 4s 9ms/step
Accuracy: 0.8351
Precision: 0.7827848101265823
Recall: 0.9276
F1-score: 0.8490617848970252


In [ ]:
#model 3
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Set the vocabulary size and sequence length
vocab_size = 10000
max_length = 200


# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping conditions
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/10
640/640 [==============================] - 90s 131ms/step - loss: 0.4452 - accuracy: 0.7973 - val_loss: 0.3789 - val_accuracy: 0.8447
Epoch 2/10
640/640 [==============================] - 40s 62ms/step - loss: 0.2480 - accuracy: 0.9092 - val_loss: 0.3600 - val_accuracy: 0.8729
Epoch 3/10
640/640 [==============================] - 31s 49ms/step - loss: 0.1658 - accuracy: 0.9443 - val_loss: 0.3541 - val_accuracy: 0.8639
Epoch 4/10
640/640 [==============================] - 26s 40ms/step - loss: 0.1378 - accuracy: 0.9552 - val_loss: 0.4273 - val_accuracy: 0.8584
Epoch 5/10
640/640 [==============================] - 24s 38ms/step - loss: 0.1236 - accuracy: 0.9574 - val_loss: 0.4677 - val_accuracy: 0.8541
Epoch 6/10
313/313 [==============================] - 6s 15ms/step - loss: 0.4738 - accuracy: 0.8506
Test accuracy: 0.851


In [ ]:
# Predict the sentiment of the test set
y_pred = model.predict(X_test)
y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

313/313 [==============================] - 5s 12ms/step
Accuracy: 0.8506
Precision: 0.8630903065451533
Recall: 0.8334
F1-score: 0.847985347985348


In [ ]:
#model 4
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Set the vocabulary size and sequence length
vocab_size = 10000
max_length = 200


# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping conditions
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/10
640/640 [==============================] - 98s 138ms/step - loss: 0.4162 - accuracy: 0.8187 - val_loss: 0.3978 - val_accuracy: 0.8443
Epoch 2/10
640/640 [==============================] - 50s 78ms/step - loss: 0.3483 - accuracy: 0.8584 - val_loss: 0.3869 - val_accuracy: 0.8393
Epoch 3/10
640/640 [==============================] - 44s 68ms/step - loss: 0.2392 - accuracy: 0.9115 - val_loss: 0.3153 - val_accuracy: 0.8648
Epoch 4/10
640/640 [==============================] - 37s 58ms/step - loss: 0.1803 - accuracy: 0.9357 - val_loss: 0.3412 - val_accuracy: 0.8646
Epoch 5/10
640/640 [==============================] - 36s 56ms/step - loss: 0.1318 - accuracy: 0.9564 - val_loss: 0.3662 - val_accuracy: 0.8633
Epoch 6/10
313/313 [==============================] - 8s 18ms/step - loss: 0.4316 - accuracy: 0.8538
Test accuracy: 0.854


In [ ]:
# Predict the sentiment of the test set
y_pred = model.predict(X_test)
y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

313/313 [==============================] - 9s 20ms/step
Accuracy: 0.8538
Precision: 0.8316460442444694
Recall: 0.8872
F1-score: 0.8585252564350686


 **Model 1:In this model we have used locally trained embeddings,coming to the model this has LSTM layer, max pooling layer and one dense layers with activation function of sigmoid respectively. Ran this modle for 10 epochs and configured early stop.This obtained an accuracy of 83.85%.**
 
 **Model 2:In this model we have used locally trained embeddings,coming to the model this has LSTM layer, max pooling layer and one dense layers with activation function of sigmoid respectively. Ran this modle for 10 epochs and configured early stop.This obtained an accuracy of 83.51%.**

**Model 3:In this model we have used locally trained embeddings,coming to the model this has LSTM layer and a dense layer with  a filter size of 1 and sigmoid activation function. Ran this modle for 10 epochs and configured early stop This obtained an accuracy of 85.06%.**

**Model 4:In this model we have used pre-rained embedding,coming to the model this has 2 LSTM layers and a dense layer with sigmoid activation. Ran this modle for 10 epochs and configured early stop. This obtained an accuracy of 85.38%.**

**By observing the above results we can say that he model 4 is performing well as the accuracy, precision , recall and F1 scores are good compared to all other models.**

**Task 6: Take away message**

Different sentiment analysis models, including Lexicon, Naive Bayes, SVM, and deep-learning models, were used in this assignment. According to this experience, Naive Bayes and Lexicon-based analysis required less time to train and obtain predictions than SVM and deep learning models. In comparison to other models, we had good results with SVM models. The given data-set produced very low performance from Lexicon. On the other hand, the performance of Naive Bayes and deep learning models on the data set was comparable.